In [1]:
####################################################
#00     I/O and Table S/U                          #
#   a- import code.                                #
#   b- associate files to variables.               #  
#   c- read input files into df's.                 #
#   d- merge df's (except error & geo (currently   #
#      only error code 0 and gecode 1 (US))).      #
####################################################

#a
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')

#b
codes_dtypes ="datain/MRTS_Dtypes.csv"
codes_months ="datain/MRTS_months.csv"
codes_cats ="datain/MRTS_cats.csv"
codes_geo ="datain/MRTS_geo.csv"
codes_err ="datain/MRTS_errors.csv"
data_in = "datain/MRTS_data.csv"      

#c
geo_df = pd.read_csv(codes_geo)     #currently only code=1(US), not merging
err_df = pd.read_csv(codes_err)     #currently no error codes, not merging 
data_df = pd.read_csv(data_in)      #val="(S)" records manually removed, all non MRTFS records
cats_df = pd.read_csv(codes_cats)
months_df = pd.read_csv(codes_months)
dtypes_df = pd.read_csv(codes_dtypes)   

#d
cats_merge = pd.merge(data_df,cats_df, how="inner", on=["cat_idx"])
mnth_merge = pd.merge(cats_merge,months_df, how="left", on=["per_idx"])
dtyp_merge = pd.merge(mnth_merge,dtypes_df, how="left", on=["dt_idx"])  


In [2]:
###########################################################################################
#01      Create Complete Dataframe                                                        #
#   a- keep only sales and perchange for non-adjusted retail.                             #
#   b- split month and year, if year not numeric, switch with month, capitalize month.    #
#    (format of per_name changes Jan/2001 (Mmm-yy --> yy-mmm))                            #
#   c- keep only records from 2010-2020.                                                  # 
#   e- complete_df: reorder columns, dropping index columns, sort, and write to csv.      #
###########################################################################################

#a
dtyp_merge =dtyp_merge[((dtyp_merge["cat_code"] == "44W72") | (dtyp_merge["cat_code"] == "4541")) &
                       ((dtyp_merge["dt_code"] == "SM") | (dtyp_merge["dt_code"] == "MPCSM")) & (dtyp_merge["is_adj"] == 0)]

#b
dtyp_merge["month"], dtyp_merge["year"] = dtyp_merge['per_name'].str.split('-', 1).str
dtyp_merge["year"] = pd.to_numeric(dtyp_merge["year"])
dtyp_merge["mon/yy"] = ""
dtyp_merge["val"].astype(float)                       # convert value field for aggregation

for x, row in dtyp_merge.iterrows():
    if dtyp_merge["month"][x].isnumeric():            # switch month and year if new format
        month_hold = dtyp_merge["year"][x]
        dtyp_merge["year"][x] = dtyp_merge["month"][x]
        dtyp_merge["month"][x] = month_hold
        dtyp_merge["month"][x] = dtyp_merge["month"][x].title()     #title all months to match old naming convention
    dtyp_merge["mon/yy"][x] = dtyp_merge["month"][x] + "/" + dtyp_merge["year"][x].astype(str)

#c
dtyp_merge = dtyp_merge[(dtyp_merge["year"] >= 10) & (dtyp_merge["year"] <= 20)]
dtyp_merge.reset_index(inplace=True,drop=True)
dtyp_merge["year"] = dtyp_merge["year"] + 2000
    
#d
complete_df = pd.DataFrame({"month": dtyp_merge["month"], "year": dtyp_merge["year"], "amount": dtyp_merge["val"],
                  "dt_unit": dtyp_merge["dt_unit"], "dt_code": dtyp_merge["dt_code"], "dt_desc": dtyp_merge["dt_desc"],
                  "cat_code": dtyp_merge["cat_code"], "cat_desc": dtyp_merge["cat_desc"],"monthyear" : dtyp_merge["mon/yy"]})

complete_df.sort_values(["cat_code", "dt_code"], inplace=True,ascending=False)
complete_df.reset_index(inplace=True, drop=True)
complete_df.to_csv("Data/censusdata_complete.csv", index=False, header=True)

In [3]:
###############################################################
#02      Monthly and Annual Summary                           #
#   a- split df into e-comm and in store for monthly sales.   #
#   b- create monthly df of sales information.                #
#   c- aggregate data and create annual summary df.           #  
#   d- write out df's to csv files.                           # 
###############################################################

#a
ecomm_sales = complete_df[(complete_df["cat_code"] == "4541") & (complete_df["dt_code"] == "SM")]
ecomm_sales.reset_index(inplace=True,drop=True)

comm_sales = complete_df[(complete_df["cat_code"] == "44W72")& (complete_df["dt_code"] == "SM")]
comm_sales.reset_index(inplace=True,drop=True)

#b
monthly_sales_both = pd.DataFrame({"month": ecomm_sales["month"], "year": ecomm_sales["year"], 
                       "ecomm salesMIL$": ecomm_sales["amount"].astype(int), "instore sales(MIL$)": comm_sales["amount"].astype(int),
                       "ecomm %":  round(ecomm_sales["amount"] / (ecomm_sales["amount"] + comm_sales["amount"]) * 100,2),
                       "total sales": (ecomm_sales["amount"] + comm_sales["amount"]).astype(int)})
#c
ecomm_annual_sales = (ecomm_sales.groupby("year")["amount"].sum()).astype(int)
comm_annual_sales = (comm_sales.groupby("year")["amount"].sum()).astype(int)
total_annual_sales = ecomm_annual_sales + comm_annual_sales
ecom_per = round((ecomm_annual_sales / total_annual_sales) * 100,2)
years = ecomm_sales.groupby('year')["year"].mean()


yearly_sales_both = pd.DataFrame({"year": years, "ecomm sales(MIL$)": ecomm_annual_sales, "instore sales(MIL$)": comm_annual_sales,
                                  "ecomm %": ecom_per, "total sales": total_annual_sales})
#d
monthly_sales_both.to_csv("Data/censusdata_monthly_sales_both.csv", index=False, header=True)
yearly_sales_both.to_csv("Data/censusdata_yearly_sales_both.csv", index=False, header=True)

In [4]:
yearly_sales_both

,year,ecomm sales(MIL$),instore sales(MIL$),ecomm %,total sales
year,,,,,
2010,2010,263488,3093706,7.85,3357194
2011,2011,294697,3251907,8.31,3546604
2012,2012,328655,3384477,8.85,3713132
2013,2013,350487,3492856,9.12,3843343
2014,2014,386065,3656015,9.55,4042080
2015,2015,433987,3811348,10.22,4245335
2016,2016,488840,3946780,11.02,4435620
2017,2017,551298,4112980,11.82,4664278
2018,2018,611743,4306377,12.44,4918120


In [5]:
monthly_sales_both

,month,year,ecomm salesMIL$,instore sales(MIL$),ecomm %,total sales
0,Jan,2010,19565,230542,7.82,250107
1,Feb,2010,18419,227215,7.50,245634
2,Mar,2010,21629,255997,7.79,277626
3,Apr,2010,20385,253668,7.44,274053
4,May,2010,20021,261194,7.12,281215
...,...,...,...,...,...,...
120,Jan,2020,56350,349654,13.88,406004
121,Feb,2020,53979,345746,13.50,399725
122,Mar,2020,62588,365250,14.63,427838
123,Apr,2020,69952,314130,18.21,384082


In [6]:
complete_df

,month,year,amount,dt_unit,dt_code,dt_desc,cat_code,cat_desc,monthyear
0,Jan,2010,19565.0,MLN$,SM,Sales - Monthly,4541,4541: Electronic Shopping and Mail-order Houses,Jan/10
1,Feb,2010,18419.0,MLN$,SM,Sales - Monthly,4541,4541: Electronic Shopping and Mail-order Houses,Feb/10
2,Mar,2010,21629.0,MLN$,SM,Sales - Monthly,4541,4541: Electronic Shopping and Mail-order Houses,Mar/10
3,Apr,2010,20385.0,MLN$,SM,Sales - Monthly,4541,4541: Electronic Shopping and Mail-order Houses,Apr/10
4,May,2010,20021.0,MLN$,SM,Sales - Monthly,4541,4541: Electronic Shopping and Mail-order Houses,May/10
...,...,...,...,...,...,...,...,...,...
495,Jan,2020,-21.9,PCT,MPCSM,Sales - Monthly Percent Change,44W72,"44W72: Retail Trade and Food Services, ex Auto...",Jan/20
496,Feb,2020,-1.1,PCT,MPCSM,Sales - Monthly Percent Change,44W72,"44W72: Retail Trade and Food Services, ex Auto...",Feb/20
497,Mar,2020,5.6,PCT,MPCSM,Sales - Monthly Percent Change,44W72,"44W72: Retail Trade and Food Services, ex Auto...",Mar/20
498,Apr,2020,-14.0,PCT,MPCSM,Sales - Monthly Percent Change,44W72,"44W72: Retail Trade and Food Services, ex Auto...",Apr/20
